In [1]:
%load_ext watermark
%watermark -a 'cs224' -u -d -v -p numpy,xarray,scipy,pandas,sklearn,matplotlib,seaborn,pymc3,gspread_pandas

/home/local/cs/local/install/anaconda3-5.3.1-Linux-x86_64/envs/py36ds/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/home/local/cs/local/install/anaconda3-5.3.1-Linux-x86_64/envs/py36ds/lib/python3.6/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


cs224 
last updated: 2020-03-15 

CPython 3.6.8
IPython 7.3.0

numpy 1.16.2
xarray 0.11.3
scipy 1.2.1
pandas 0.24.2
sklearn 0.20.3
matplotlib 3.0.3
seaborn 0.9.0
pymc3 3.8
gspread_pandas 2.1.2


In [2]:
%matplotlib inline
import numpy as np, scipy, scipy.stats as stats, scipy.special, scipy.misc, pandas as pd, matplotlib.pyplot as plt, seaborn as sns, xarray as xr
import matplotlib as mpl, io, requests, gspread_pandas

import pymc3 as pm

import theano as thno
import theano.tensor as T

import datetime, time, math
from dateutil import relativedelta

from collections import OrderedDict

SEED = 42
np.random.seed(SEED)

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
# pd.set_option('display.float_format', lambda x: '%.2f' % x)
np.set_printoptions(edgeitems=10)
np.set_printoptions(linewidth=1000)
np.set_printoptions(suppress=True)
np.core.arrayprint._line_width = 180

sns.set()

In [3]:
from IPython.display import display, HTML

from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        if type(df) == np.ndarray:
            df = pd.DataFrame(df)
        html_str+=df.to_html()
    html_str = html_str.replace('table','table style="display:inline"')
    # print(html_str)
    display_html(html_str,raw=True)

CSS = """
.output {
    flex-direction: row;
}
"""

def display_graphs_side_by_side(*args):
    html_str='<table><tr>'
    for g in args:
        html_str += '<td>'
        html_str += g._repr_svg_()
        html_str += '</td>'
    html_str += '</tr></table>'
    display_html(html_str,raw=True)
    

display(HTML("<style>.container { width:70% !important; }</style>"))

In [4]:
%load_ext autoreload
%autoreload 1
%aimport covid19

* [Novel Corona Virus 2019 Dataset](https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset)
    * [COVID19_line_list_data.csv](https://docs.google.com/spreadsheets/d/e/2PACX-1vQU0SIALScXx8VXDX7yKNKWWPKE1YjFlWc6VTEVSN45CklWWf-uWmprQIyLtoPDA18tX9cFDr-aQ9S6/pubhtml)
    * [COVID_open_line_list_data.csv](https://docs.google.com/spreadsheets/d/1itaohdPiAeniCXNlntNztZ_oRvjh0HsGuJXUJWET008/edit#gid=0)


* [Getting Google Spreadsheet CSV into A Pandas Dataframe](https://stackoverflow.com/questions/19611729/getting-google-spreadsheet-csv-into-a-pandas-dataframe)
* [How to access Google Sheet data using the Python API and convert to Pandas dataframe](https://towardsdatascience.com/how-to-access-google-sheet-data-using-the-python-api-and-convert-to-pandas-dataframe-5ec020564f0e)
* [gspread-pandas](https://pypi.org/project/gspread-pandas/)
* [Reading google sheets Into a Pandas Dataframe with gspread and OAuth2](https://medium.com/@vince.shields913/reading-google-sheets-into-a-pandas-dataframe-with-gspread-and-oauth2-375b932be7bf)
* [New Google Spreadsheets publish limitation](https://stackoverflow.com/questions/21189665/new-google-spreadsheets-publish-limitation)
* [Google spreadsheet direct download link for only ONE sheet as excel](https://stackoverflow.com/questions/27000699/google-spreadsheet-direct-download-link-for-only-one-sheet-as-excel)

```
conda install -c conda-forge gspread-pandas
```

In [5]:
google_docs_source = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQU0SIALScXx8VXDX7yKNKWWPKE1YjFlWc6VTEVSN45CklWWf-uWmprQIyLtoPDA18tX9cFDr-aQ9S6/pubhtml'

In [6]:
# gs = gspread_pandas.Spread(google_docs_source)
# gs

In [7]:
columns = ['case_in_country', 'reporting date', 'e1', 'summary', 'location', 'country', 'gender', 'age', 'symptom_onset', 'If_onset_approximated',
    'hosp_visit_date', 'exposure_start', 'exposure_end', 'visiting Wuhan', 'from Wuhan', 'death', 'recovered', 'symptom', 'source', 'link']


df = pd.read_csv('COVID19_line_list_data.csv', index_col=0, parse_dates=['reporting date','symptom_onset', 'hosp_visit_date', 'exposure_start', 'exposure_end'])
df.drop(columns=list(df.columns)[len(columns):],inplace=True)
#c = list(df.columns)[]
#df.columns = columns
df.head()

,case_in_country,reporting date,Unnamed: 3,summary,location,country,gender,age,symptom_onset,If_onset_approximated,hosp_visit_date,exposure_start,exposure_end,visiting Wuhan,from Wuhan,death,recovered,symptom,source,link
id,,,,,,,,,,,,,,,,,,,,
1,NaN,2020-01-20,NaN,First confirmed imported COVID-19 pneumonia pa...,"Shenzhen, Guangdong",China,male,66.0,2020-01-03,0.0,2020-01-11,2019-12-29,2020-01-04,1,0.0,0,0,NaN,Shenzhen Municipal Health Commission,http://wjw.sz.gov.cn/wzx/202001/t20200120_1898...
2,NaN,2020-01-20,NaN,First confirmed imported COVID-19 pneumonia pa...,Shanghai,China,female,56.0,2020-01-15,0.0,2020-01-15,NaT,2020-01-12,0,1.0,0,0,NaN,Official Weibo of Shanghai Municipal Health Co...,https://www.weibo.com/2372649470/IqogQhgfa?fro...
3,NaN,2020-01-21,NaN,First confirmed imported cases in Zhejiang: pa...,Zhejiang,China,male,46.0,2020-01-04,0.0,2020-01-17,NaT,2020-01-03,0,1.0,0,0,NaN,Health Commission of Zhejiang Province,http://www.zjwjw.gov.cn/art/2020/1/21/art_1202...
4,NaN,2020-01-21,NaN,new confirmed imported COVID-19 pneumonia in T...,Tianjin,China,female,60.0,NaT,NaN,2020-01-19,NaT,NaT,1,0.0,0,0,NaN,人民日报官方微博,https://m.weibo.cn/status/4463235401268457?
5,NaN,2020-01-21,NaN,new confirmed imported COVID-19 pneumonia in T...,Tianjin,China,male,58.0,NaT,NaN,2020-01-14,NaT,NaT,0,0.0,0,0,NaN,人民日报官方微博,https://m.weibo.cn/status/4463235401268457?


In [8]:
df.country.unique()

array(['China', 'France', 'Japan', 'Malaysia', 'Nepal', 'Singapore', 'South Korea', 'Taiwan', 'Thailand', 'USA', 'Vietnam', 'Australia', 'Canada', 'Cambodia', 'Sri Lanka', 'Germany', 'UAE', 'Hong Kong', 'Italy', 'Russia', 'UK', 'India', 'Phillipines', 'Finland', 'Spain', 'Sweden', 'Belgium', 'Egypt', 'Iran', 'Israel', 'Lebanon', 'Kuwait', 'Bahrain', 'Austria', 'Afghanistan', 'Algeria', 'Croatia', 'Switzerland'], dtype=object)

In [9]:
len(df[df.country == 'Germany'])

54

In [10]:
df = pd.read_csv('COVID19_2020_open_line_list - outside_Hubei.csv', index_col=0, parse_dates=['date_onset_symptoms', 'date_admission_hospital', 'date_confirmation', 'travel_history_dates', 'date_death_or_discharge'])
df.head()

/home/local/cs/local/install/anaconda3-5.3.1-Linux-x86_64/envs/py36ds/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (0,17,22,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,age,sex,city,province,country,wuhan(0)_not_wuhan(1),latitude,longitude,geo_resolution,date_onset_symptoms,date_admission_hospital,date_confirmation,symptoms,lives_in_Wuhan,travel_history_dates,travel_history_location,reported_market_exposure,additional_information,chronic_disease_binary,chronic_disease,source,sequence_available,outcome,date_death_or_discharge,notes_for_discussion,location,admin3,admin2,admin1,country_new,admin_id,data_moderator_initials
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,30,male,"Chaohu City, Hefei City",Anhui,China,1.0,31.646960,117.716600,admin3,18.01.2020,20.01.2020,22.01.2020,NaN,yes,17.01.2020,Wuhan,NaN,NaN,NaN,NaN,http://ah.people.com.cn/GB/n2/2020/0127/c35826...,NaN,NaN,NaT,NaN,NaN,Chaohu City,Hefei City,Anhui,China,340181,NaN
2,47,male,"Baohe District, Hefei City",Anhui,China,1.0,31.778630,117.331900,admin3,10.01.2020,21.01.2020,23.01.2020,NaN,no,10.01.2020,"Luzhou Hunan, via Wuhan",NaN,NaN,NaN,NaN,http://ah.people.com.cn/GB/n2/2020/0127/c35826...,NaN,NaN,NaT,NaN,NaN,Baohe District,Hefei City,Anhui,China,340111,NaN
3,49,male,"High-Tech Zone, Hefei City",Anhui,China,1.0,31.828313,117.224844,point,15.01.2020,20.01.2020,23.01.2020,NaN,no,10.01.2020,"Yinzhou Hunan, via Wuhan",NaN,NaN,NaN,NaN,http://ah.people.com.cn/GB/n2/2020/0127/c35826...,NaN,NaN,NaT,NaN,High-Tech Zone,Shushan District,Hefei City,Anhui,China,340104,NaN
4,47,female,"High-Tech Zone, Hefei City",Anhui,China,1.0,31.828313,117.224844,point,17.01.2020,20.01.2020,23.01.2020,NaN,no,nan,NaN,NaN,contacted with confirmed case,NaN,NaN,http://ah.people.com.cn/GB/n2/2020/0127/c35826...,NaN,NaN,NaT,NaN,High-Tech Zone,Shushan District,Hefei City,Anhui,China,340104,NaN
5,50,female,"Feidong County, Hefei City",Anhui,China,1.0,32.001230,117.568100,admin3,10.01.2020,21.01.2020,23.01.2020,NaN,no,07.01.2020,Wuhan,NaN,"06.01.2020 went to Wuhan, 07.01.2020 returned ...",NaN,NaN,http://ah.people.com.cn/GB/n2/2020/0127/c35826...,NaN,NaN,NaT,NaN,NaN,Feidong County,Hefei City,Anhui,China,340122,NaN


In [11]:
ldf = df[df.country == 'Germany']
ldf

,age,sex,city,province,country,wuhan(0)_not_wuhan(1),latitude,longitude,geo_resolution,date_onset_symptoms,date_admission_hospital,date_confirmation,symptoms,lives_in_Wuhan,travel_history_dates,travel_history_location,reported_market_exposure,additional_information,chronic_disease_binary,chronic_disease,source,sequence_available,outcome,date_death_or_discharge,notes_for_discussion,location,admin3,admin2,admin1,country_new,admin_id,data_moderator_initials
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1485,33,male,Starnberg,Bavaria,Germany,1.0,47.997220,11.340560,point,24.01.2020,nan,27.01.2020,"chills, fever, myalgias, sore throat",no,nan,NaN,no,Infection acquired locally from colleague from...,0.0,NaN,(1) https://www.nejm.org/doi/full/10.1056/NEJM...,NaN,discharged,2020-01-27,NaN,Starnberg,NaN,NaN,Bavaria,Germany,2,NaN
1880,27-40,male,Starnberg,Bavaria,Germany,1.0,47.997220,11.340560,point,24.01.2020,nan,28.01.2020,NaN,no,nan,NaN,no,Had contact with colleague from Shanghai atten...,0.0,NaN,(1) https://www.nejm.org/doi/full/10.1056/NEJM...,NaN,NaN,NaT,NaN,Starnberg,NaN,NaN,Bavaria,Germany,2,NaN
1881,27-40,male,Starnberg,Bavaria,Germany,1.0,47.997220,11.340560,point,26.01.2020,nan,28.01.2020,NaN,no,nan,NaN,no,Had contact with colleauge (ID 1481) between 2...,0.0,NaN,(1) https://www.nejm.org/doi/full/10.1056/NEJM...,NaN,NaN,NaT,NaN,Starnberg,NaN,NaN,Bavaria,Germany,2,NaN
1882,33,female,Starnberg,Bavaria,Germany,1.0,47.997220,11.340560,point,26.01.2020,nan,28.01.2020,NaN,no,nan,NaN,no,Had contact with colleage (ID 1481) between 21...,0.0,NaN,(1) https://www.nejm.org/doi/full/10.1056/NEJM...,NaN,NaN,NaT,NaN,Starnberg,NaN,NaN,Bavaria,Germany,2,NaN
4107,NaN,male,Traunstein,Bavaria,Germany,1.0,47.866670,12.633330,point,nan,nan,30.01.2020,NaN,no,nan,NaN,NaN,Worked in same company as the other four cases,NaN,NaN,https://www.stmgp.bayern.de/presse/aktuelle-in...,NaN,NaN,NaT,NaN,Traunstein,NaN,NaN,Bavaria,Germany,2,NaN
4108,NaN,NaN,Traunstein,Bavaria,Germany,1.0,47.866670,12.633330,point,nan,nan,31.01.2020,NaN,no,nan,NaN,NaN,Child of the first Traunstein case,NaN,NaN,https://www.stmgp.bayern.de/presse/aktuelle-in...,NaN,NaN,NaT,NaN,Traunstein,NaN,NaN,Bavaria,Germany,2,NaN
4862,33,male,Munich,Bavaria,Germany,1.0,48.133330,11.566670,point,nan,nan,02.02.2020,NaN,NaN,nan,NaN,NaN,Employee of company where six other cases were...,NaN,NaN,https://www.stmgp.bayern.de/presse/aktuelle-in...,NaN,NaN,NaT,NaN,Munich,NaN,NaN,Bavaria,Germany,2,NaN
4863,52,male,Fuerstenfeldbruck,Bavaria,Germany,1.0,48.177780,11.255560,point,nan,nan,31.01.2020,NaN,NaN,nan,NaN,NaN,Employee of company where six other cases were...,NaN,NaN,https://www.stmgp.bayern.de/presse/aktuelle-in...,NaN,NaN,NaT,NaN,Fuerstenfeldbruck,NaN,NaN,Bavaria,Germany,2,NaN
4864,NaN,NaN,Germersheim,Rhineland-Palatinate,Germany,1.0,49.216670,8.366667,point,nan,nan,02.02.2020,NaN,NaN,01.02.2020,Wuhan,NaN,NaN,NaN,NaN,https://www.kreis-germersheim.de/kv_germershei...,NaN,released from quarantine,2020-02-16,NaN,Germersheim,NaN,NaN,Rhineland-Palatinate,Germany,11,NaN


In [12]:
df.outcome.unique()

array([nan, 'discharged', 'discharge', 'died', 'Discharged', 'death', 'Symptoms only improved with cough. Currently hospitalized for follow-up.', 'released from quarantine', 'recovered', 'stable', 'critical condition, intubated as of 14.02.2020', 'treated in an intensive care unit (14.02.2020)', 'severe', 'recovering at home 03.03.2020', 'not hospitalized'], dtype=object)

In [13]:
ldf[['city','province', 'date_confirmation', 'outcome', 'date_death_or_discharge']]

,city,province,date_confirmation,outcome,date_death_or_discharge
ID,,,,,
1485,Starnberg,Bavaria,27.01.2020,discharged,2020-01-27
1880,Starnberg,Bavaria,28.01.2020,NaN,NaT
1881,Starnberg,Bavaria,28.01.2020,NaN,NaT
1882,Starnberg,Bavaria,28.01.2020,NaN,NaT
4107,Traunstein,Bavaria,30.01.2020,NaN,NaT
4108,Traunstein,Bavaria,31.01.2020,NaN,NaT
4862,Munich,Bavaria,02.02.2020,NaN,NaT
4863,Fuerstenfeldbruck,Bavaria,31.01.2020,NaN,NaT
4864,Germersheim,Rhineland-Palatinate,02.02.2020,released from quarantine,2020-02-16
